In [44]:
from tabulate import tabulate
import sqlite3
import json

In [45]:
from pydantic import BaseModel

class InstalledMCP(BaseModel):
    name: str
    clients: list[str]
    read_access: list[str]
    write_access: list[str]
    model: str | None = None
    host: str
    managed_by: str
    proxy: str | None = None
    verified: bool
    
    def format_as_tabulate_row(self) -> list[str]:
        return [
            self.name,
            ", ".join(self.clients),
            ", ".join(self.read_access),
            ", ".join(self.write_access),
            self.model if self.model is not None else "",
            self.host,
            self.managed_by,
            self.proxy,
            "y" if self.verified else "n",
        ]
        
    @classmethod
    def from_db_row(cls, row: sqlite3.Row):
        row = dict(row)
        row['clients'] = json.loads(row['clients'])
        row['read_access'] = json.loads(row['read_access'])
        row['write_access'] = json.loads(row['write_access'])
        return cls(**row)

In [71]:
INSTALLED_HEADERS = ["NAME", "CLIENTS", "READ_ACCESS", "WRITE_ACCESS", "MODEL", "HOST", "MANAGED_BY", "PROXY", "VERIFIED"]

In [72]:
# conn = sqlite3.connect("mcp.db")
conn = sqlite3.connect(":memory:")
conn.row_factory = sqlite3.Row
curr = conn.cursor()

def create_table(conn: sqlite3.Connection):
    curr = conn.cursor()
    curr.execute("CREATE TABLE IF NOT EXISTS mcps (name TEXT, clients TEXT, read_access TEXT, write_access TEXT, model TEXT, host TEXT, managed_by TEXT, proxy TEXT, verified TEXT)")

def insert_mcp(conn: sqlite3.Connection, mcp: InstalledMCP):
    curr = conn.cursor()
    curr.execute(f"INSERT INTO mcps ({','.join(INSTALLED_HEADERS)}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
        (mcp.name, 
         json.dumps(mcp.clients), 
         json.dumps(mcp.read_access), 
         json.dumps(mcp.write_access), 
         mcp.model, 
         mcp.host, 
         mcp.managed_by, 
         mcp.proxy, 
         mcp.verified))
    # curr.execute("INSERT INTO mcps (name, clients, read_access, write_access, model, host, managed_by, proxy, verified) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", (mcp.name, mcp.clients, mcp.read_access, mcp.write_access, mcp.model, mcp.host, mcp.managed_by, mcp.proxy, mcp.verified))

def query_mcps(conn: sqlite3.Connection):
    curr = conn.cursor()
    curr.execute("SELECT * FROM mcps")
    return curr.fetchall()

def get_mcps(conn: sqlite3.Connection):
    res = query_mcps(conn)
    return [InstalledMCP.from_db_row(row) for row in res]
    
def get_mcp(conn: sqlite3.Connection, name: str):
    curr = conn.cursor()
    curr.execute("SELECT * FROM mcps WHERE name = ?", (name,))
    row = curr.fetchone()
    return InstalledMCP.from_db_row(row)

create_table(conn)
# conn.close()

In [73]:
def install(name: str, clients: list[str], read_access: list[str], write_access: list[str], model: str | None = None, host: str | None = None, managed_by: str | None = None, proxy: str | None = None, verified: bool = False):
    mcp = InstalledMCP(
        name=name,
        clients=clients,
        read_access=read_access,
        write_access=write_access,
        model=model,
        host=host,
        managed_by=managed_by,
        proxy=proxy,
        verified=verified,
    )
    insert_mcp(conn, mcp)
    
    # launch mcp server. 3 options: 
    # 1. local mcp server over stdio
    # 2. local mcp server over http (not for now?) -> requires daemon
    # 3. remote openmined mcp server over http
    

In [ ]:
import json
import platform
from pathlib import Path

class MCPConfigItem(BaseModel):
    name: str
    json_body: dict
    client: str
    

def current_claude_desktop_config():
    if platform.system() != 'Darwin':
        raise RuntimeError("Currently only macOS is supported")
    HOME = Path.home()
    with open(f"{HOME}/Library/Application Support/Claude/claude_desktop_config.json", "r") as f:
        return json.load(f)

def get_claude_config_items():
    full_json = current_claude_desktop_config()
    for item in full_json['mcpServers']:
        yield MCPConfigItem(
            name=item['name'],
            json_body=item['jsonBody'],
            client="claude",
        )

# TODO
**list mcp servers needed for demo**:
- github
- custom notes mcp over streamable http?
- screenpipe (only download )


**design daemon framework**:
- skip first, just check before every command if
- 1 there were manual changes to mcp servers (for stdio, proxies)
- 

**toolbox daemon responsibilities:**
- start at startup
- for all local http servers, make sure they are on
- perhaps update the state? what if someone manually installed an mcp in cursor




- implement installation for demo mcp servers
- implement daemon such that they are always online

In [74]:
install(name="github mcp", clients=["claude", "cursor"],
        read_access=["Issues", "PRs", "Settings"], write_access=["Issues", "PRs", "Settings"],
        model="", host="localhost", managed_by="macbook-pro-koen", proxy=None, verified=True)

In [78]:
def installed(conn):
    mcps = get_mcps(conn)
    print(
        tabulate(
            [m.format_as_tabulate_row() for m in mcps],
            headers=INSTALLED_HEADERS,
            tablefmt='orgtbl'
            )
        )
    

In [ ]:
installed(conn)